In [49]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from xgboost import XGBClassifier

%matplotlib inline

In [39]:
df = pd.read_csv('./data/df_mod.csv', index_col = 0)

In [40]:
df_test = df.drop(labels=['Cat', 'fixed', 'male'], axis=1)

In [41]:
X = df_test.iloc[:, :-1]

In [42]:
y = df_test.iloc[:, -1]

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y)

In [46]:
rfc = RandomForestClassifier(n_estimators = 250, n_jobs = -1)

scores = cross_val_score(rfc, X_train, y_train, scoring='f1_micro', cv = 3)

In [47]:
scores

array([0.61132929, 0.61409356, 0.6117626 ])

In [ ]:
xg = XGBClassifier()

scores = cross_val_score(xg, X_train, y_train, scoring='f1_micro', cv=3)

In [ ]:
scores